In [17]:
import pandas as pd 
from ast import literal_eval
from collections import Counter
df = pd.read_csv("./scrapped_data/1_Rock_2020.csv").drop(columns = "Unnamed: 0")
list_of_chords= [literal_eval(l) for l in list(df.chords)]
all_chords = [chord for l in list_of_chords for chord in l]

In [25]:
Counter(all_chords).keys()

dict_keys(['Am', 'D', 'G', 'C', 'Cadd9', 'E7', 'Dm7', 'Dm7b5', 'Am/G', 'D/F#', 'G7', 'F', 'Fm', 'Em', 'Fmaj7', 'G#m', 'F#', 'C#m', 'B', 'E', 'D#', 'C5', 'E5', 'A5', 'F5', 'G5', 'A', 'E/G#', 'Amaj7', 'Am7', 'F#m', 'Dsus2', 'DM7', 'A7', 'Em7', 'G#', 'F#m11', 'Bm', 'Gm', 'Bb', 'Dm', 'Bbm', 'Cmaj7', 'Gmaj7', 'B7', 'G#7', 'C/E', 'C7/E', 'C7', 'Bbmaj7', 'Fm7', 'Cm7', 'Dbmaj7', 'Ab', 'Bbm7', 'D7', 'C#dim', 'Em7b5', 'Cdim', 'Cdim7', 'F#7', 'A/C#', 'Csus2', 'Cm', 'A6', 'C#7', 'Bm7', 'Gm7', 'Badd4', 'C/G', 'D/A', 'F#m7', 'F#m6', 'Dmaj7', 'C#+', 'C#m7', 'C#7+5', 'E6', 'Bm9', 'E6/G#', 'F#madd9', 'D5', 'F7', 'Emaj7', 'Bdim7', 'Adim'])

In [ ]:
C#
F#
Bb
Ab
Db